In [ ]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-ma

# BERT fine tuning con testa di classificazione per Next Sentence Prediction

Questo file è stato utilizzato solo su Colab per poter sfruttare le GPU, tutti i path quindi si riferiscono a quella posizione, e non funzionano su questa cartella. Non sono stati cambiati e non è stato rieseguito in locale per non cambiare lo stato del notebook dell'addestramento del modello usato per i test.

In [ ]:
import os
import numpy as np
import pandas as pd
import datasets
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForNextSentencePrediction
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# nome modello
model_name = 'dbmdz/bert-base-italian-cased'

# carico il modello
model = AutoModelForNextSentencePrediction.from_pretrained(model_name, num_labels=2)

# tokenizzatore associato
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

In [ ]:
# in questo caso carico direttamente i dataset, che ho estratto nel notebook "bert_fine_tuning_coherence_tedx"
train = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_train')
val = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_val')
test = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_test')

In [ ]:
test

Dataset({
    features: ['id', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 800
})

In [ ]:
config_dict = model.config

In [ ]:
model.cls

BertOnlyNSPHead(
  (seq_relationship): Linear(in_features=768, out_features=2, bias=True)
)

In [ ]:

num_epochs = 5



training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    save_strategy = "epoch",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
def acc_metric(eval_pred):
  accuracy_metric = evaluate.load("accuracy")
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)

  return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
# fine-tuning del modello
trainer = Trainer(
    model,
    training_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
    compute_metrics=acc_metric
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.654700,0.589084,0.690000
2,0.526900,0.595901,0.702500
3,0.371900,0.750559,0.678750
4,0.262300,0.958734,0.673750
5,0.190800,1.375597,0.678750


TrainOutput(global_step=2250, training_loss=0.4013322991265191, metrics={'train_runtime': 1539.491, 'train_samples_per_second': 23.384, 'train_steps_per_second': 1.462, 'total_flos': 7887732328653120.0, 'train_loss': 0.4013322991265191, 'epoch': 5.0})

In [ ]:
# salvo il modello
trainer.save_model("/content/drive/My Drive/FINETUNED_MODEL_NSP")

In [ ]:
log_history = trainer.state.log_history

In [ ]:
# salvo i logs
import json
with open('/content/drive/My Drive/finetuned_model_NSP_logs.json', 'w') as f:
    json.dump(log_history, f)